In [1]:
if 0:
  %pip uninstall ale-py -y
  %pip install ale-py

  %pip install -U gym>=0.24.0
  %pip install -U gym[atari,accept-rom-license]

In [2]:
import cv2
import gym
import gym.spaces
import numpy as np
import collections
import importlib

import time
import argparse
import numpy as np

import torch

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:424: UserWarning: WARN: Custom namespace `ALE` is being overridden by namespace `ALE`. If you are developing a plugin you shouldn't specify a namespace in `register` calls. The namespace is specified through the entry point package metadata.
  f"Custom namespace `{spec.namespace}` is being overridden "


In [3]:
gym.__version__

'0.24.0'

In [4]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [5]:
mypath = "drive.MyDrive.Colab Notebooks.Hands_on_Deep_RL.Chapter06.lib"

In [6]:
dqn_model=importlib.import_module(mypath+'.dqn_model')
wrappers = importlib.import_module(mypath+'.wrappers')

In [7]:
DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
FPS = 25

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument("-m", "--model", required=True,
                    help="Model file to load")
parser.add_argument("-e", "--env", default=DEFAULT_ENV_NAME,
                    help="Environment name to use, default=" +
                          DEFAULT_ENV_NAME)
parser.add_argument("-r", "--record", help="Directory for video")
parser.add_argument("--no-vis", default=True, dest='vis',
                    help="Disable visualization",
                    action='store_false')

_StoreFalseAction(option_strings=['--no-vis'], dest='vis', nargs=0, const=False, default=True, type=None, choices=None, help='Disable visualization', metavar=None)

In [9]:
model_path = '/content/drive/MyDrive/Colab Notebooks/Hands_on_Deep_RL/Chapter06/best_19.dat'

In [10]:
args = parser.parse_args(['--m',model_path, '--r', 'my_record', '--no-vis'])
args

Namespace(env='PongNoFrameskip-v4', model='/content/drive/MyDrive/Colab Notebooks/Hands_on_Deep_RL/Chapter06/best_19.dat', record='my_record', vis=False)

In [11]:
if 1:
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
  !apt-get install x11-utils > /dev/null 2>&1 
  !pip install pyglet > /dev/null 2>&1 
  !apt-get install -y xvfb python-opengl > /dev/null 2>&1
  !pip install gym pyvirtualdisplay > /dev/null 2>&1

In [14]:
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

In [12]:
env = wrappers.make_env(args.env)

In [13]:

if args.record:
    env = gym.wrappers.RecordVideo(env, args.record)
    #env = gym.wrappers.Monitor(env, directory="mon", force=True)
    #env = gym.wrappers.Monitor(env, directory=args.record, force=True)
    pass

net = dqn_model.DQN(env.observation_space.shape,
                    env.action_space.n)
state = torch.load(args.model, map_location=lambda stg, _: stg)
net.load_state_dict(state)

state = env.reset()
total_reward = 0.0
c = collections.Counter()

In [14]:
env.reset()
while True:
    start_ts = time.time()
    
    env.render(mode='rgb_array')
    state_v = torch.tensor(np.array([state], copy=False))
    q_vals = net(state_v).data.numpy()[0]
    action = np.argmax(q_vals)
    c[action] += 1
    state, reward, done, _ = env.step(action)
    total_reward += reward
    if done:
        break

print("Total reward: %.2f" % total_reward)
print("Action counts:", c)
if args.record:
    env.env.close()

Total reward: 21.00
Action counts: Counter({5: 422, 0: 369, 1: 232, 2: 219, 3: 206, 4: 187})


In [18]:
!dir recvideo

rl-video-episode-0.meta.json  rl-video-episode-0.mp4


In [19]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('./recvideo/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)